In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore, iqr
from sklearn.model_selection import train_test_split
import numpy as np
import missingno as msno
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data = pd.read_csv("smoking.csv")


In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
for col, dtype in data.dtypes.items():
    print(f"Column '{col}' has data type: {dtype}")

In [ ]:
print(data.isnull().values.any())

In [ ]:
columns_to_assign = ['hear_left', 'hear_right', 'SMK_stat_type_cd', 'urine_protein']
data_num = data.select_dtypes(include='number')
data_num.drop(columns_to_assign, axis=1, inplace=True)



data_cat = data.select_dtypes(exclude='number')
data_cat[columns_to_assign] = data[columns_to_assign]



In [ ]:
unique_values_all = {col: data_cat[col].unique() for col in data_cat.columns}

print(unique_values_all)


In [ ]:
data.isnull().sum()

In [ ]:
def visualData(data, time):
   for col in data.columns:
        print(col)
     # Plot Histogram of Z-scores
        plt.figure(figsize=(12, 5))

        # Histogram plot
        plt.subplot(1, 2, 1)
        plt.hist(data[col], bins=100, edgecolor='k', alpha=0.7)
        plt.title(f'Histogram of values {col}')
        plt.xlabel('value')
        plt.ylabel('Frequency')
        plt.legend()

        # Scatter Plot with Z-scores
        plt.subplot(1, 2, 2)
        plt.scatter(time, data[col], alpha=0.7)
        plt.title(f'Scatter Plot of values {col}')
        plt.xlabel('time')
        plt.ylabel('value')
        plt.grid(True)

        plt.tight_layout()
        plt.show()

In [ ]:
visualData(data=data_num, time=data_num.index)

In [ ]:

normal_columns=['DBP', 'SBP', 'tot_chole', 'hemoglobin', 'weight', 'age', 'height', 'waistline']
skewed_columns=[ 'BLDS', 'HDL_chole', 'LDL_chole', 'triglyceride', 'SGOT_AST', 'SGOT_ALT', 'gamma_GTP', 'serum_creatinine', 'sight_left', 'sight_right']


In [ ]:
def Zscore(data):
# Calculate Z-scores for each data point
    z_scores = zscore(data)
    # Define a threshold for identifying outliers
    threshold = 3
    # Identify outliers
    outliers = (z_scores > threshold) | (z_scores < -threshold)
    return outliers, z_scores

z_outliers, zscores = Zscore(data_num[normal_columns])



In [ ]:
def visualZscore_2(data, z_scores, outliers, threshold=3):
    """
    Visualize actual values instead of Z-scores for a dataset.
    
    Parameters:
    - data: Original DataFrame with actual values.
    - z_scores: DataFrame with Z-scores for each column.
    - outliers: DataFrame or boolean mask of outliers for each column.
    - threshold: Z-score threshold for outliers.
    """
    for col in z_scores.columns:
        # Calculate mean and std deviation of the original data
        mean = data[col].mean()
        std = data[col].std()

        # Convert Z-score thresholds back to actual value thresholds
        lower_bound = mean - threshold * std
        upper_bound = mean + threshold * std

        # Plot Histogram of Actual Values
        plt.figure(figsize=(12, 5))

        # Histogram plot
        plt.subplot(1, 2, 1)
        plt.hist(data[col], bins=100, edgecolor='k', alpha=0.7)
        plt.axvline(lower_bound, color='r', linestyle='dashed', linewidth=1.5, label=f'Outlier Threshold ({lower_bound:.2f})')
        plt.axvline(upper_bound, color='r', linestyle='dashed', linewidth=1.5, label=f'Outlier Threshold ({upper_bound:.2f})')
        plt.title(f'Histogram of Actual Values ({col})')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.yscale('log')
        plt.legend()

        # Scatter Plot with Actual Values
        plt.subplot(1, 2, 2)
        plt.scatter(range(len(data[col])), data[col], c=['red' if outlier else 'blue' for outlier in outliers[col]], alpha=0.7)
        plt.axhline(lower_bound, color='r', linestyle='dashed', linewidth=1.5, label=f'Lower Bound ({lower_bound:.2f})')
        plt.axhline(upper_bound, color='r', linestyle='dashed', linewidth=1.5, label=f'Upper Bound ({upper_bound:.2f})')
        plt.title(f'Scatter Plot of Actual Values ({col})')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.grid(True)
        plt.legend()

        # Highlight outliers
        """for i, (value, is_outlier) in enumerate(zip(data[col], outliers[col])):
            if is_outlier:
                plt.text(i, value, f'{value:.2f}', fontsize=9, color='red', ha='left', va='bottom')"""

        plt.tight_layout()
        plt.show()


In [ ]:
def visualZscore(z_scores, outliers, threshold=3):
    for col in z_scores.columns:
        # Plot Histogram of Z-scores
        plt.figure(figsize=(12, 5))

        # Histogram plot
        plt.subplot(1, 2, 1)
        plt.hist(z_scores[col], bins=100, edgecolor='k', alpha=0.7)
        plt.axvline(threshold, color='r', linestyle='dashed', linewidth=1.5, label=f'Outlier Threshold (+{threshold})')
        plt.axvline(-threshold, color='r', linestyle='dashed', linewidth=1.5, label=f'Outlier Threshold (-{threshold})')
        plt.title(f'Histogram of Z-scores {col}')
        plt.xlabel('Z-score')
        plt.ylabel('Frequency')
        plt.yscale('log')
        plt.legend()

        # Scatter Plot with Z-scores
        plt.subplot(1, 2, 2)
        plt.scatter(range(len(z_scores[col])), z_scores[col], c=['red' if outlier else 'blue' for outlier in outliers[col]], alpha=0.7)
        plt.axhline(threshold, color='r', linestyle='dashed', linewidth=1.5)
        plt.axhline(-threshold, color='r', linestyle='dashed', linewidth=1.5)
        plt.title(f'Scatter Plot of Z-scores {col}')
        plt.xlabel('Index')
        plt.ylabel('Z-score')
        plt.grid(True)

        

        # Highlight outliers
        for i, (score, is_outlier) in enumerate(zip(z_scores[col], outliers)):
            if is_outlier:
                plt.text(i, score, f'{score:.2f}', fontsize=9, color='red', ha='left', va='bottom')

        plt.tight_layout()
        plt.show()

In [ ]:
def IQR(df):
    summary = df.describe()
    Q1 = summary.loc['25%']
    Q3 = summary.loc['75%']
    IQR = Q3 - Q1

    lower_bounds = Q1 - 1.5 * IQR
    upper_bounds = Q3 + 1.5 * IQR
    outliers_dict = {}
    
    for feature in df.columns:
        lower_bound = lower_bounds[feature]
        upper_bound = upper_bounds[feature]
        
        # Identify outliers
        outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
        outliers_dict[feature] = outliers
        
    return outliers_dict, lower_bounds, upper_bounds

iqr, lower_bound, upper_bound = IQR(data_num[skewed_columns])


In [ ]:
def visualIQR(data, lower_bound, upper_bound):
    for col in data.columns:
        # Plot Histogram of Z-scores
        plt.figure(figsize=(12, 5))

        # Histogram plot
        plt.subplot(1, 2, 1)
        plt.hist(data[col], bins=100, color='lightblue', edgecolor='black', alpha=0.7)
        plt.axvline(x=lower_bound[col], color='r', linestyle='--', label=f'Lower Bound ({lower_bound[col]})')
        plt.axvline(x=upper_bound[col], color='g', linestyle='--', label=f'Upper Bound ({upper_bound[col]})')

        # Customize plot
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.yscale('log')
        plt.title(f'Histogram with IQR Boundaries {col}')
        plt.legend()

        # Scatter Plot with Z-scores
        plt.subplot(1, 2, 2)
        plt.scatter(range(len(data[col])), data[col], label='Data Points', color='b')
        plt.axhline(y=lower_bound[col], color='r', linestyle='--', label=f'Lower Bound ({lower_bound[col]})')
        plt.axhline(y=upper_bound[col], color='g', linestyle='--', label=f'Upper Bound ({upper_bound[col]})')

        # Marking outliers
        outliers = data[(data[col] < lower_bound[col]) | (data[col] > upper_bound[col])]
        plt.scatter(np.where((data[col] < lower_bound[col]) | (data[col] > upper_bound[col]))[0], outliers[col], color='r', label='Outliers')

        # Customize plot
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.title(f'Scatter Plot with IQR Boundaries and Outliers {col}')
        plt.legend()
        


        plt.tight_layout()
        plt.show()

In [ ]:
visualIQR(data=data_num[skewed_columns], lower_bound=lower_bound, upper_bound=upper_bound)

In [ ]:
visualZscore(z_scores=zscores, outliers=z_outliers)

In [ ]:
visualZscore_2(data=data_num[normal_columns], z_scores=zscores, outliers=z_outliers)

In [ ]:
#Handling outliers
data = data[data['SGOT_ALT'] >= 3]
data = data[data['SGOT_AST'] >= 3] 
data = data[data['SGOT_ALT'] <= 1000] 
data = data[data['SGOT_AST'] <= 1000]  
data = data[data['BLDS'] >= 40] 
data = data[data['BLDS'] <= 400] 
data = data[data['HDL_chole'] >= 16] 
data = data[data['waistline'] >= 30] 
data = data[data['waistline'] <= 200] 
data = data[data['serum_creatinine'] >= 0.25] 
data = data[data['tot_chole'] >= 80]
data = data[data['DBP'] >= 46.38] 
data = data[data['SBP'] >= 78.8] 
data = data[data['hemoglobin'] >= 9.48] 

data['gamma_GTP'] = np.where(data['gamma_GTP'] > 400, 400, data['gamma_GTP'])  # Cap high values
data['triglyceride'] = np.where(data['triglyceride'] > 500, 500, data['triglyceride'])
data['HDL_chole'] = np.where(data['HDL_chole'] > 96, 96, data['HDL_chole'])
data['LDL_chole'] = np.where(data['LDL_chole'] > 204, 204, data['LDL_chole'])
data['sight_left'] = np.where(data['sight_left'] > 1.95, 1.95, data['sight_left'])
data['sight_right'] = np.where(data['sight_right'] > 1.95, 1.95, data['sight_right'])
data['serum_creatinine'] = np.where(data['serum_creatinine'] > 20, 20, data['serum_creatinine'])
data['tot_chole'] = np.where(data['tot_chole'] > 500, 500, data['tot_chole'])
data['DBP'] = np.where(data['DBP'] > 105.72, 105.72, data['DBP'])
data['SBP'] = np.where(data['SBP'] > 166, 166, data['SBP'])
data['hemoglobin'] = np.where(data['hemoglobin'] > 18.98, 18.98, data['hemoglobin'])

In [ ]:
data_num = data.select_dtypes(include='number')
data_num.drop(columns_to_assign, axis=1, inplace=True)
visualData(data=data_num, time=data_num.index)

In [ ]:
#Data transformation

#Feature scaling

skew_features = ['gamma_GTP', 'SGOT_ALT', 'SGOT_AST', 'triglyceride']
reflect_features=['sight_left', 'sight_right']



In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Create a copy of the original data
data_transformed = data_num.copy()

# Reflect and log-transform reflect_features
for feature in reflect_features:
    if data_transformed[feature].min() >= 0:
        # Reflect the data
        data_transformed[feature] = data_transformed[feature].max() + 1 - data_transformed[feature]
        # Apply log transformation
        data_transformed[feature] = np.log1p(data_transformed[feature])
    else:
        print(f"Feature {feature} has negative values and cannot be reflected.")

# Log-transform skew_features
for feature in skew_features:
    data_transformed[feature] = np.log1p(data_transformed[feature])

# Apply Min-Max Scaling to the entire DataFrame
scaler = MinMaxScaler()
data_transformed_scaled = scaler.fit_transform(data_transformed)

# Convert the scaled NumPy array back to a DataFrame (optional)
data_transformed_scaled = pd.DataFrame(data_transformed_scaled, columns=data_transformed.columns)

# Check results
data_transformed_scaled.head()

In [ ]:

ord_features = ['hear_right', 'hear_left', 'urine_protein', 'SMK_stat_type_cd']
notord_features=['sex', 'DRK_YN']

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for feature in ord_features:
    le = LabelEncoder()
    data_cat[feature] = le.fit_transform(data_cat[feature])
    label_encoders[feature] = le  # Save the encoder for later inverse transformation if needed

# One-Hot Encoding for Non-Ordinal Features
data_cat = pd.get_dummies(data_cat, columns=notord_features, drop_first=True)  # drop_first avoids multicollinearity
data_cat = data_cat.astype(int)
data_cat.head()

In [ ]:
#visualData(data=data_transformed_scaled, time=data_transformed_scaled.index)

In [ ]:
# Avoid division by zero by replacing zeros in the denominator
epsilon = 1e-10  # A small constant to prevent division by zero

# Waist-to-Weight Ratio
data_transformed_scaled['waist_to_weight_ratio'] = (
    data_transformed_scaled['waistline'] / (data_transformed_scaled['weight'] + epsilon)
)

# Pulse Pressure
data_transformed_scaled['pulse_pressure'] = (
    data_transformed_scaled['SBP'] - data_transformed_scaled['DBP']
)

# Cholesterol Ratio
data_transformed_scaled['cholesterol_ratio'] = (
    data_transformed_scaled['tot_chole'] / (data_transformed_scaled['HDL_chole'] + epsilon)
)

# AST/ALT Ratio
data_transformed_scaled['ast_alt_ratio'] = (
    data_transformed_scaled['SGOT_AST'] / (data_transformed_scaled['SGOT_ALT'] + epsilon)
)

In [ ]:
data_cat_aligned = data_cat.loc[data_transformed_scaled.index]

# Combine the aligned data
data_new = pd.concat([data_cat_aligned, data_transformed_scaled], axis=1)


In [ ]:
y = data_new['SMK_stat_type_cd']
data_new.drop('SMK_stat_type_cd', axis=1, inplace=True)



In [ ]:
data_new.head()

In [ ]:
y.head()

In [ ]:
# Check for NaN values
print("Are there NaN values?")
print(data_new.isna().sum())

# Check for Infinite values
print("Are there infinite values?")
print(np.isinf(data_new).sum())


In [ ]:
#PCA
from sklearn.decomposition import PCA
pca = PCA()
principal_components = pca.fit_transform(data_new)

# Convert to DataFrame
pca_df = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(principal_components.shape[1])])

print(pca_df.head())
# Explained variance
explained_variance = pca.explained_variance_ratio_

# Cumulative explained variance
cumulative_variance = np.cumsum(explained_variance)

print("Explained Variance by Component:", explained_variance)
print("Cumulative Explained Variance:", cumulative_variance)
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(explained_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Scree Plot')
plt.grid()
plt.show()

In [ ]:
pca = PCA(n_components=3)
df_pca = pca.fit(data_new)


plt.figure(figsize=(9,5))
plt.scatter(x=[i+1 for i in range(len(df_pca.explained_variance_ratio_))], y=df_pca.explained_variance_ratio_, s=200, alpha=0.75, c='red')
plt.grid(True)
plt.title("Explained variance ratio of the \nfitted principal component vector\n")
plt.xlabel("Principal components", fontsize=15)
plt.xticks([i+1 for i in range(len(df_pca.explained_variance_ratio_))], fontsize=13)
plt.yticks(fontsize=15)
plt.ylabel("Varaiance ratio", fontsize=15)
# plt.show()

df_trans = pca.transform(data_new)
df_trans = pd.DataFrame(data=df_trans)
df_combined = pd.concat([data_new, df_trans, y], axis=1)
# print(df_combined.head(10))
columns1=df_combined.columns

In [ ]:
pca = PCA(n_components=3)
principal_components = pca.fit_transform(data_new)

# Convert to DataFrame
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3'])

pca_df.head()

In [ ]:
scaler = MinMaxScaler()
pca_minmax_scaled = scaler.fit_transform(principal_components)

# Convert to DataFrame
pca_minmax_df = pd.DataFrame(pca_minmax_scaled, columns=['PC1', 'PC2', 'PC3'])
pca_minmax_df.head()

In [ ]:
data_proc = pd.concat([pca_minmax_df, data_new], axis=1)
data_test = pd.concat([data_proc, y], axis=1)

In [ ]:
#feature extraction
import seaborn as sns
correlation_matrix = data_test.corr()
plt.figure(figsize=(16, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Features and target
X = data_test[['PC1', 'PC2', 'PC3', 'sex_Male', 'DRK_YN_Y']]  # Feature columns

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the splits
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)



In [ ]:
X_train.to_csv('train/X_train_cluster.csv', index=False)
y_train.to_csv('train/y_train_cluster.csv', index=False)

X_test.to_csv('test/X_test_cluster.csv', index=False)
y_test.to_csv('test/y_test_cluster.csv', index=False)